# Introduction

The following uses a kind of literate programming approach to build a library of tools useful for writing unit and integration tests directly into a notebook. The library is to be articulated as a Python package built as the concatenation of a subset of the code cells of this notebook, using an ad hoc script. To help with identifying which code cells are parts of the final package and which are inline testing code, we use *tags*, which make up cell metadata in this notebook.

In [1]:
%load_ext pycodestyle_magic
%flake8_on --max_line_length 120 --ignore E302

In [2]:
from abc import ABC, abstractmethod
from copy import copy
from inspect import getframeinfo, Traceback, unwrap
from io import TextIOBase
import itertools
from linecache import getline
import operator
import re
import sys
from traceback import walk_tb
from typing import Dict, List, Tuple, Iterator, Union, Iterable, Optional, Any, Callable, Mapping, Sequence, cast,\
        Container

import colors
from IPython import get_ipython
from IPython.core.magic import register_cell_magic
from pygments import highlight
from pygments.lexers import Python3Lexer
from pygments.formatters import TerminalFormatter

# Test results

In [3]:
class Result(ABC):
    """
    Result of a test. Indicates whether the test passed (was a success), and if it did not,
    whether it was a failure (as opposed to any other kind of issue).
    """

    @abstractmethod
    def is_success(self) -> bool:
        """True when an associated test run has passed."""
        raise NotImplementedError()

    def is_failure(self) -> bool:
        """True when an associated has not passed because a designed failure condition was met."""
        return False

    def as_dict(self) -> Dict:
        """Expresses this result as a dictionary suitable to structured data serialization."""
        return {"type": type(self).__name__}

## Manual test failure

Tests can be made to fail deliberately by raising a special exception.

In [4]:
class TestFailed(Exception):
    """
    Exception raised by this framework in order to mark a test run as a Failure.
    """

    def __init__(self, reason: str) -> None:
        super().__init__(reason)
        self.reason = reason

In [5]:
try:
    raise TestFailed("asdf")
except TestFailed as err:
    assert str(err) == "asdf"

In [6]:
def fail(reason: str = ""):
    "Marks some ongoing test as failed, with an optional reason for failure."
    raise TestFailed(reason)

In [7]:
try:
    fail("asdf")
    assert False
except TestFailed as err:
    assert err.reason == "asdf"

## Test result: success

In [8]:
class Success(Result):
    """
    Result for a test that passed.
    """
    def is_success(self) -> bool:
        return True

In [9]:
assert Success().is_success()

In [10]:
assert Success().as_dict() == {"type": "Success"}

## Test result following the test code raising an exception

### Traceback frames

In [11]:
class Frame:
    """
    Information regarding a frame of a traceback. Provides more than the very limited
    code context that comes from standard library introspection tools.
    """

    def __init__(self, tb: Traceback, num_line: int, tags: Optional[List[str]] = None) -> None:
        self.num_line = num_line
        self.name_file = tb.filename
        self.function = "" if tb.function == "<module>" else tb.function
        self.tags = tags or []

    def name_file_pretty(self, capitalized: bool = False) -> str:
        m = re.match(r"<ipython-input-(\d+)-[a-f0-9A-F]+>", self.name_file)
        if m:
            name = f"code cell {m.group(1)}"
            return name.capitalize() if capitalized else name
        return self.name_file

    def context(self, before: int = 3, after: int = 3) -> List[Tuple[int, str]]:
        ctx = [(self.num_line, getline(self.name_file, self.num_line).rstrip())]
        for delta in range(1, before + 1):
            ctx.insert(0, (self.num_line - delta, getline(self.name_file, self.num_line - delta).rstrip()))
        for delta in range(1, after + 1):
            ctx.append((self.num_line + delta, getline(self.name_file, self.num_line + delta).rstrip()))

        # Clean up context: remove line-ending blanks and blank lines top and bottom
        # of the context blob.
        while len(ctx) > 0:
            for i in [0, -1]:
                if len(ctx[i][1]) == 0:
                    del ctx[i]
                    break
            else:
                break

        return ctx

    def as_dict(self, context_before: int = 3, context_after: int = 3) -> Dict:
        return {
            "file": self.name_file_pretty(False),
            "line": self.num_line,
            "function": self.function,
            "context": [[i, line] for i, line in self.context(context_before, context_after)],
            "tags": self.tags
        }

    def __str__(self) -> str:
        return f"{self.function + ': ' if self.function else ''}File {self.name_file}, Line {self.num_line}"

    def __repr__(self) -> str:
        return str(self)

In [12]:
from inspect import getfile

def my_function():  # noqa
    getfile("asdf")  # Will raise a TypeError

frames = []  # noqa
try:
    my_function()
    assert False
except TypeError:
    _, _, tb = sys.exc_info()
    for frame_raw, lineno in walk_tb(tb):
        frame = Frame(getframeinfo(frame_raw), lineno)
        assert frame.name_file == getfile(frame_raw)
        assert frame.num_line == lineno

        if frame_raw.f_code.co_name == "<module>":
            assert frame.function == ""
        else:
            assert frame.function == frame_raw.f_code.co_name

        assert frame.tags == []
        frames.append(frame)

assert len(frames) == 3

In [13]:
frame = frames[1]
assert frame.context(0, 0) == [(4, "    getfile(\"asdf\")  # Will raise a TypeError")]
assert frame.context(1, 1) == [(3, "def my_function():  # noqa"), (4, "    getfile(\"asdf\")  # Will raise a TypeError")]
assert frame.context(3, 3) == [
    (1, "from inspect import getfile"),
    (2, ""),
    (3, "def my_function():  # noqa"),
    (4, "    getfile(\"asdf\")  # Will raise a TypeError"),
    (5, ""),
    (6, "frames = []  # noqa"),
    (7, "try:")
]

In [14]:
assert frame.context(45, 0) == list(zip(range(1, 4 + 1), [
    "from inspect import getfile",
    "",
    "def my_function():  # noqa",
    "    getfile(\"asdf\")  # Will raise a TypeError"
]))

In [15]:
assert frame.context(0, 9000) == list(zip(range(4, 9000), """\
    getfile("asdf")  # Will raise a TypeError

frames = []  # noqa
try:
    my_function()
    assert False
except TypeError:
    _, _, tb = sys.exc_info()
    for frame_raw, lineno in walk_tb(tb):
        frame = Frame(getframeinfo(frame_raw), lineno)
        assert frame.name_file == getfile(frame_raw)
        assert frame.num_line == lineno

        if frame_raw.f_code.co_name == "<module>":
            assert frame.function == ""
        else:
            assert frame.function == frame_raw.f_code.co_name

        assert frame.tags == []
        frames.append(frame)

assert len(frames) == 3\
""".split("\n")))

In [16]:
assert frame.as_dict() == {
    "file": "code cell 12",
    "line": 4,
    "function": "my_function",
    "tags": [],
    "context": list(list(e) for e in zip(range(1, 7 + 1), """\
from inspect import getfile

def my_function():  # noqa
    getfile("asdf")  # Will raise a TypeError

frames = []  # noqa
try:\
""".split("\n")))
}

### The exception-driven result: errors

In [17]:
class Error(Result):
    """
    Non-passing test result due to an exception being raised.

    It is passed a set of common functions: the presence of these functions in the
    traceback of the exception are expected and normal, making their eventual
    reporting redundant and sort of trivial. The frames corresponding to these functions
    in the traceback summary kept by this object will be tagged as such.
    """
    TAG_COMMON = "common"

    def __init__(self, fns_common: Iterable[Callable]) -> None:
        super().__init__()
        self._type_exc: type
        self._value_exc: Any
        self._type_exc, self._value_exc, tb = sys.exc_info()
        if tb is None:
            raise RuntimeError("Can only instantiate this class when an exception has been raised.")

        codes_common = {unwrap(fn).__code__ for fn in fns_common}
        self._traceback: List[Frame] = []
        for frame_raw, num_line in walk_tb(tb):
            tags = []
            if frame_raw.f_code in codes_common:
                tags.append(Error.TAG_COMMON)
            self._traceback.append(Frame(getframeinfo(frame_raw), num_line, tags))

    def is_success(self) -> bool:
        return False

    @property
    def type_exc(self) -> type:
        """Returns the type of the exception associated to this result."""
        return self._type_exc

    @property
    def value_exc(self) -> Any:
        """Returns the exception raised in association to this test result."""
        return self._value_exc

    @property
    def traceback(self) -> List[Frame]:
        """
        Returns a summary of the stack trace associated to the exception that brought this test result.
        """
        return self._traceback

    def as_dict(self, context_before: int = 3, context_after: int = 3) -> Dict:
        d = super().as_dict()
        d.update(
            {
                "type_exc": self.type_exc.__name__,
                "value_exc": str(self.value_exc),
                "traceback": [frame.as_dict(context_before, context_after) for frame in self.traceback]
            }
        )
        return d

In [18]:
from inspect import getfile


def fn_raise():
    raise RuntimeError()


def caller():
    fn_raise()


try:
    caller()
    assert False
except RuntimeError:
    err: Error = Error([caller])
    assert not err.is_success()
    assert not err.is_failure()
    assert err.type_exc == RuntimeError
    assert isinstance(err.value_exc, RuntimeError)
    assert len(err.traceback) == 3
    assert [frame.function for frame in err.traceback] == ["", "caller", "fn_raise"]
    assert [frame.tags for frame in err.traceback] == [[], [Error.TAG_COMMON], []]

In [19]:
try:
    raise RuntimeError()
    assert False
except RuntimeError:
    assert {
        "type": "Error",
        "type_exc": "RuntimeError",
        "value_exc": "",
        "traceback": [
            {
                "file": "code cell 19",
                "line": 2,
                "function": "",
                "tags": [],
                "context": [[2, "    raise RuntimeError()"]]
            }
        ]
    } == Error([]).as_dict(0, 0)

### Deliberate exception: failures

For convenience's sake, we model `Failure`s as a subclass of `Error` to gain the exception breakdown functionality.

In [20]:
NameArg = Union[int, str]
Arg = Tuple[NameArg, Any]
Args = Sequence[Arg]


def join_args(args: Sequence[Any], kwargs: Mapping[str, Any]) -> Args:
    return list(enumerate(args)) + sorted(list(kwargs.items()), key=lambda arg: arg[0])

In [21]:
class Explanation:
    """
    Embodies the reason why a predicate fails, including a main expectation that was not satisfied and
    a list of assertion arguments that tripped the failure.
    """

    def __init__(self, main: str, culprits: Sequence[Union[Arg, Tuple[str]]]) -> None:
        self.main = main
        self.culprits = culprits

    def __repr__(self) -> str:
        return f"{type(self).__name__}({repr(self.main)}, {repr(self.culprits)})"

    def __bool__(self) -> bool:
        """
        All explanations are for failures, so when a predicate returns an explanation, it's as if it had
        returned False.
        """
        return False

    def as_dict(self) -> Dict:
        return {
            "main": self.main,
            "culprits": [[name, str(value)] for name, value in self.culprits]
        }

In [22]:
class DetailedAssertionError(AssertionError):

    def __init__(self, explanation: Explanation):
        super().__init__(explanation.main)
        self.explanation = explanation

In [23]:
class Failure(Error):
    """
    Test result stemming from a condition check that failed, or a test run marked
    as a failure.
    """
    def __init__(self, explanation: Explanation, fns_common: Iterable[Callable]):
        super().__init__(fns_common)
        self._explanation = explanation

    @property
    def explanation(self) -> Explanation:
        "Reason given by the programmer as to why the test failed."
        return self._explanation

    @property
    def reason(self) -> str:
        reason = self.explanation.main
        if self.explanation.culprits:
            reason += ": " + ", ".join(str(value) for _, value in self.explanation.culprits)
        return reason

    def is_failure(self) -> bool:
        return True

    def as_dict(self, context_before: int = 3, context_after: int = 3) -> Dict:
        d = super().as_dict(context_before, context_after)
        d["explanation"] = self.explanation.as_dict()
        return d

In [24]:
try:
    assert False
except AssertionError:
    err: Failure = Failure(Explanation("asdf", [("a", 1234), ("c", 987)]), [])
    assert not err.is_success()
    assert err.is_failure()
    assert err.type_exc == AssertionError
    assert isinstance(err.value_exc, AssertionError)
    assert isinstance(err.traceback, list)
    assert err.reason == "asdf: 1234, 987"

In [25]:
try:
    assert False
except AssertionError:
    assert {
        "type": "Failure",
        "type_exc": "AssertionError",
        "value_exc": "",
        "traceback": [
            {
                "file": "code cell 25",
                "line": 2,
                "function": "",
                "tags": [],
                "context": [[2, "    assert False"]]
            }
        ],
        "explanation": {
            "main": "asdf",
            "culprits": [[0, "1234"], [1, "3456"], ["qwer", "6767"]]
        }
    } == Failure(Explanation("asdf", [(0, 1234), (1, 3456), ("qwer", 6767)]), []).as_dict(0, 0)

# Custom assertions to get detailed failure descriptions

The tool `pytest` goes to great lengths to decompose the abstract syntax tree involved in an assertion in order to provide to the user a detailed description of, say, a failed equality test. While stealing this work would be ideal, the short-term solution rather is to provide custom assert functions that divide comparands into distinct function arguments.

## The main assertion verb

In [26]:
ExplanationOnFailure = Union[bool, str, Explanation]
Predicate = Callable[..., ExplanationOnFailure]


def assert_(predicate: Predicate, *ops: Any, msg: str = "", **kwops: Any) -> None:
    """
    Evalutes a condition over multiple objects, using as condition test either an explicit function predicate
    or a bound method provided as first (left-most) operand. Raises an AssertionError exception if the condition
    is not met (the condition test yields a result of False).

    cond
        Predicate function (or bound method). It must take as input parameters. The function should return True
        to signal satisfaction of the predicate. In the other case, it may return a False-equivalent value (in which
        case the failure explanation is provided as the `msg` keyword argument), or a string corresponding to the
        explanation, or an ExplanationFailure instance.
    msg
        Facultative explanation for the failure of satisfying the predicate. If provided, it will override the
        explanation returned by the predicate, or at least its "main expectation" (see docstring of
        `ExplanationFailure`).
    ops, kwops
        Operands to the predicate.
    """
    result = predicate(*ops, **kwops)
    if not result:
        if isinstance(result, Explanation):
            explanation = result
        else:
            explanation = Explanation(result or "", join_args(ops, kwops))

        if msg:
            explanation.main = msg
        raise DetailedAssertionError(explanation)

In [27]:
# Assertions driven by simple functions.
assert_((1).__eq__, 1)
try:
    assert_(operator.eq, 5, 7)
    raise RuntimeError("The last assertion should have raised!")
except AssertionError as err:
    assert isinstance(err, DetailedAssertionError)
    assert not err.explanation.main
    assert err.explanation.culprits == [(0, 5), (1, 7)]

In [28]:
# Assertion with explicit message and keyword arguments.


def eq(v, x=0, y=0):
    return v == x and x == y


try:
    assert_(eq, 0, x=0, y=1, msg="Ho")
except AssertionError as err:
    assert isinstance(err, DetailedAssertionError)
    assert err.explanation.main == "Ho"
    assert str(err) == "Ho"
    assert err.explanation.culprits == [(0, 0), ("x", 0), ("y", 1)]

## Useful predicates

In [29]:
def same(*args, **kwargs) -> ExplanationOnFailure:
    """
    Predicate satisfied when all its arguments are the same (according to operator `is`).
    """
    return _assess_sequence_2more(
        "These objects are not the same as {name} => {value}",
        lambda p, r: None if p is r else hex(hash(p)),
        args,
        kwargs
    )

In [30]:
def _assess_sequence_2more(
    expl_main: str,
    pred_repr: Callable[[Any], Any],
    args: Iterator[Any],
    kwargs: Mapping[str, Any]
) -> ExplanationOnFailure:
    try:
        (name_ref, value_ref), *rest = join_args(args, kwargs)
        culprits = [(name_ref, pred_repr(value_ref, None))]
        for name_comp, value_comp in rest:
            r = pred_repr(value_comp, value_ref)
            if r is not None:
                culprits.append((name_comp, r))

        if len(culprits) > 1:
            return Explanation(expl_main, culprits)
        return True
    except ValueError:
        raise ValueError("Should provide at least one test argument to this function.")

In [31]:
o1 = object()
o2 = object()

assert same(o1, o1)
assert same(o2, o2, o2, a=o2)

expl = same(o1, o2)
assert not expl
assert "not the same" in expl.main
assert expl.culprits == [(0, hex(hash(o1))), (1, hex(hash(o2)))]

In [32]:
# Not providing enough arguments.
o1 = object()

try:
    same()
    assert False, "Should have raised ValueError"
except ValueError:
    pass

In [33]:
def eq(*args: Any, **kwargs: Any) -> ExplanationOnFailure:
    """
    Exact equality predicate.
    """
    return _assess_sequence_2more(
        "These objects are not equal",
        lambda p, r: None if p == r else p,
        args,
        kwargs
    )

In [34]:
assert eq(1, 1, 1)

expl = eq(0, 0, 0, 1)
assert "not equal" in expl.main
assert expl.culprits == [(0, 0), (3, 1)]

assert not eq(1, 0)

try:
    eq()
    assert False, "Function should have raised"
except ValueError:
    pass

In [35]:
def diff(*args: Any, **kwargs: Any) -> ExplanationOnFailure:
    """
    Difference predicate.
    """
    all_args = join_args(args, kwargs)
    culprits_by_value: List[Tuple[arg, Set[str]]] = []
    for i, (name_ref, value_ref) in enumerate(all_args[:-1]):
        for name_comp, value_comp in all_args[i + 1:]:
            if value_ref == value_comp:
                try:
                    names_equal = [s for v, s in culprits_by_value if v == value_ref][0]
                except IndexError:
                    culprits_by_value.append((value_ref, set()))
                    _, names_equal = culprits_by_value[-1]
                names_equal |= {str(n) for n in [name_ref, name_comp]}

    if len(culprits_by_value) > 0:
        return Explanation(
            "Some of the arguments were not mutually different",
            sorted([(", ".join(sorted(list(names))), value) for value, names in culprits_by_value])
        )
    return True

In [36]:
# Difference assertion.
assert diff(0, 1)
assert diff(1, 8, 5)
assert diff(5, "asdf")
assert diff(0, None)

assert not diff(1, 1)

expl = diff(1, 1, 5)
assert "not mutually different" in expl.main
assert expl.culprits == [("0, 1", 1)]

expl = diff(1, 5, 5)
assert expl.culprits == [("1, 2", 5)]

expl = diff(1, 5, 8, 9, 9, n=9, w=5)
assert expl.culprits == [("1, w", 5), ("3, 4, n", 9)]

assert diff()

In [37]:
def between(lower: Any, upper: Any, bounds: str = "[]") -> Predicate:
    """
    Returns a predicate that is satisfied if all its arguments fall within an interval. Comparands
    must all support order comparison operators (<, >, <=, >=).

    lower
        Lower bound of the interval.
    upper
        Upper bound of the interval.
    bounds
        Determines whether the interval is closed or open at either end (e.g. the boundary element is belongs or not
        to the set formed by the interval). It is provided as a string whose first and last characters are observed.
            First character:
                [    Interval is closed on the left.
                (    Interval is open on the left.
            Last character:
                ]    Interval is closed on the right.
                )    Interval is open on the right.
        Any other character in these positions raise an error.
    """
    if lower > upper:
        raise ValueError("Lower bound should be lesser than the upper bound.")
    cmp_lower, cmp_upper = _get_comparators(bounds)
    return _assessor_sequence(
        f"These objects did not belong to interval {bounds[0]}{lower}, {upper}{bounds[1]}",
        lambda v: cmp_lower(v, lower) and cmp_upper(v, upper),
    )


Comparator = Callable[[Any, Any], bool]


def _get_comparators(bounds: str) -> Tuple[Comparator, Comparator]:
    if not re.match(r"^(\[|\().*(\]|\))$", bounds):
        raise ValueError("Invalid open/close bounds descriptor")
    return (
        {"[": operator.ge, "(": operator.gt}[bounds[0]],
        {"]": operator.le, ")": operator.lt}[bounds[-1]]
    )

In [38]:
def _assessor_sequence(expl_main: str, pred: Callable[[Any], bool]) -> Predicate:
    def _assess_sequence(*args, **kwargs) -> ExplanationOnFailure:
        culprits = [(name, value) for name, value in join_args(args, kwargs) if not pred(value)]
        if len(culprits) > 0:
            return Explanation(expl_main, culprits)
        return True
    return _assess_sequence

In [39]:
assert between(1, 2)(1.38, 1 + 1e-12, 2 - 1e-12)

expl = between(1, 2)(1 - 1e-12, 2 + 1e-12)
assert "belong to interval" in expl.main
assert [n for n, _ in expl.culprits] == [0, 1]

assert between(1, 2, bounds="[asdf]")(1, a=2)
assert between(1, 2, bounds="[)")(1, 2 - 1e-15)
assert not between(1, 2, bounds="(]")(1)
assert between(1, 2, bounds="(]")(1 + 1e-15, 2)
assert not between(1, 2, bounds="[])")(2)
assert not between(1, 2, bounds="(())")(1, 2)
assert between(1, 2, bounds="()")(1 + 1e-15, 2 - 1e-15)

assert between(1, 1)(1)

In [40]:
# between: lower bound must be smaller or equal than upper bound
try:
    between(3, 2)(2.5)
    assert False, "Should have raised"
except ValueError:
    pass

In [41]:
# between: many ways to bork the bound closedness descriptor
for b in ["", "[", "][", ")", ")("]:
    try:
        between(1, 2, bounds=b)
        raise RuntimeError("Last call should have raised")
    except ValueError:
        pass

In [42]:
def approx(centre: Any, tol: Any = 1e-6, bounds: str = "[]") -> Predicate:
    """
    Returns a predicate satisfied when all arguments land within an interval defined by a centre and a tolerance
    (which corresponds to its half-width). This can be used with any number-like objects that support addition and
    some form of subtraction (either direct or by addition with opposite).

    centre
        Reference number against which we test for near-equality.
    tol
        Acceptable difference between arguments and the centre for the predicate to be satisfied.
    bounds
        Exclusivity of each implicit interval boundaries; see the docstring for `between` for further details.
    """
    upper = centre + tol
    try:
        lower = centre - tol
    except TypeError:
        try:
            lower = centre + (-tol)
        except TypeError:
            raise TypeError("Operands do not support any form of subtraction")

    return between(lower, upper, bounds=bounds)

In [43]:
assert approx(1, tol=1e-3)(1 + 9e-4, 1 - 9.99e-4)
assert approx(1, tol=1e-6)(1 - 1e-6, 1.000001)
assert not approx(1, tol=1e-6, bounds="()")(1 - 1e-6, 1.000001)

In [44]:
def belong_to(collection: Container) -> Predicate:
    """
    Predicate satisfied when all arguments are contained by the given collection, according to the `in'
    operator.
    """
    return _assessor_sequence("These arguments did not belong to the collection", lambda v: v in collection)

In [45]:
assert belong_to([1, 2, 3])(3, q=2)
assert not belong_to([1, 3])(2)
assert belong_to([1, 2, 3])()

In [46]:
def is_any_of(*types: type) -> Predicate:
    """
    Predicate satisfied when all arguments are instances of one of the given types.
    """
    if len(types) < 1:
        raise ValueError("Must provide at least one type.")
    return _assessor_sequence(
        "These arguments were not of type" + " or ".join(T.__name__ for T in types),
        lambda v: any(isinstance(v, T) for T in types)
    )

In [47]:
assert is_any_of(int)(a=6, b=-92)
assert is_any_of(float)(9.2)
assert not is_any_of(float)(9.2, 3)
assert is_any_of(int, float)(9.2, 3)

# Test suites

In [48]:
class Subscriber(ABC):
    """
    Object reacting to test results as they are generated by running tests.
    """

    def on_add_to_suite(self, suite: "Suite") -> None:
        pass

    def on_result(self, name_test: str, result: Result) -> None:
        pass

In [49]:
TestFunction = Callable[..., None]


class Suite:
    """
    Suite of tests, gathering the result of multiple named test runs. Test code fragments
    are named using the `test()` decorator, or leveraging it indirectly by registering
    a shortcut cell magic.

    Test suites gets added functionality through a publish/subscribe system. Subscriber are
    special objects tied to the suite instance through its `|' (bit OR) operator. At the
    moment, the only event broadcast to all subscribers is the generation of a new test
    result (and its appending to the suite's log). For instance, the `Report` plug-in
    supports the suite by giving immediate feedback on a test's results. Thus, to
    instantiate a suite with this added feature, one would use code like

    suite = Suite() | Report()
    """

    def __init__(self, name_magic: str = "test") -> None:
        self._tests: Dict[str, List[Result]] = {}
        self._fns_common = [fail, self.test]
        self._subscribers: List[Subscriber] = []

    def test(
        self,
        fn: Optional[TestFunction] = None,
        name: str = "",
        args: Sequence[Any] = [],
        kwargs: Mapping[str, Any] = {}
    ) -> Union[Callable[[TestFunction], TestFunction], TestFunction]:
        """
        Runs a test encoded into a function. Completing the function's execution counts as a
        test success; tripped assertions and other exceptions generate some other Result;
        and the test result is retained by this Suite instance.

        This decorator can be used two ways. Without application, one can decorate a
        test function without parameter:

            @suite.test
            def this_is_my_test():
                # Test goes here!

        The name of the test corresponds to that of the function. Applying the decorator can
        supply arguments to the test function and override the name of the test.

            @suite.test(name="My test, with spaces", args=(3, 4))
            def fn_test(a, b):
                # Test goes here!

        To run a test with multiple parameter sets, one may even call this function directly,
        not as a decorator:

            def fn_test(a, b):
                # Test test test...

            for a, b in [(2, 8), (3, 4)]:
                suite.test(fn_test, name=f"Test with {a}, {b}", args=(a, b))

        fn
            Function that embodies the test code.
        name
            Name of the test; by default, the name of the function is used.
        args
            Positional arguments to pass to the function to run the test.
        kwargs
            Named arguments to pass to the function to run the test.
        """
        if fn is None:
            return lambda fn: self.test(fn, name=name, args=args, kwargs=kwargs)

        try:
            fn(*args, **kwargs)
            result = Success()
        except TestFailed as err:
            result = Failure(err.reason, self._fns_common)
        except AssertionError as err:
            result = Failure(str(err), self._fns_common)
        except BaseException:
            result = Error(self._fns_common)

        name_test = name
        if not name_test:
            name_test = fn.__name__
            if args or kwargs:
                str_args = ", ".join(
                    [repr(str(a)) for a in args] +
                    [f"{k}={repr(str(v))}" for k, v in kwargs.items()]
                )
                name_test += f"({str_args})"
        self._tests.setdefault(name_test, []).append(result)
        for subscriber in self._subscribers:
            subscriber.on_result(name_test, result)

        return fn

    @property
    def results(self) -> Iterator[Tuple[str, Iterator[Result]]]:
        """
        Iterates through the gathered test results. For each named test, yields a tuple of
        the name of the test and an iterator over each result gathered as the test has
        been run.
        """
        for name, test_results in self._tests.items():
            yield name, iter(test_results)

    def as_dict(self) -> Dict[str, List[Dict]]:
        "Provides a structured data representation suitable for data serialization and exportation."
        return {name: [r.as_dict() for r in rez] for name, rez in self.results}

    def __or__(self, subscriber: Subscriber) -> "Suite":
        """
        Generates a clone of this suite instance, but with this subscriber subscribed to it.

        The new suite will not share member data structures with `self`, but if `self` carries
        test results already, the new suite will reference the same result objects -- we
        assume that Result objects are immutable.
        """
        suite_with_subscriber = Suite()
        suite_with_subscriber._tests = copy(self._tests)  # Under assumption of results immutability.
        suite_with_subscriber._subscribers = copy(self._subscribers)
        suite_with_subscriber._subscribers.append(subscriber)
        subscriber.on_add_to_suite(suite_with_subscriber)
        return suite_with_subscriber

In [50]:
assert isinstance(Suite()._tests, dict)

In [51]:
suite = Suite()


@suite.test
def succeeding():
    assert True


@suite.test
def failing_by_assert_terse():
    assert False


@suite.test
def failing_by_assert_reason():
    assert False, "assert reason"


@suite.test
def failing_manually_terse():
    fail()


@suite.test
def failing_manually_reason():
    fail("fail reason")


@suite.test
def error():
    raise RuntimeError("doh")


assert [
    ("succeeding", [(Success, "")]),
    ("failing_by_assert_terse", [(Failure, "")]),
    ("failing_by_assert_reason", [(Failure, "assert reason")]),
    ("failing_manually_terse", [(Failure, "")]),
    ("failing_manually_reason", [(Failure, "fail reason")]),
    ("error", [(Error, "")])
] == [(name, [(type(r), r.reason if hasattr(r, "reason") else "") for r in rez]) for name, rez in suite.results]

In [52]:
# Check each non-success for tagging of the first frame.
num_non_success = 0
for name, rez in suite.results:
    for r in rez:
        if not r.is_success():
            num_non_success += 1
            assert Error.TAG_COMMON in r.traceback[0].tags

assert num_non_success == 5

In [53]:
# Check failures for tagging of the fail call.
num_failures = 0
for name, rez in suite.results:
    if "failing_manually" in name:
        for r in rez:
            num_failures += 1
            assert Error.TAG_COMMON in r.traceback[-1].tags

assert num_failures == 2

In [54]:
suite = Suite()


@suite.test(name="trial")
def t1():
    fail()


@suite.test(name="trial")
def t2():
    raise RuntimeError()


@suite.test(name="trial")
def t3():
    pass  # Literally!


assert [("trial", [Failure, Error, Success])] == [(name, [type(r) for r in rez]) for name, rez in suite.results]

In [55]:
suite = Suite()


@suite.test
def first():
    fail()


@suite.test(name="first")
def first2():
    pass


@suite.test
def second():
    raise RuntimeError()


assert {name: [r["type"] for r in rez] for name, rez in suite.as_dict().items()} == {
    "first": ["Failure", "Success"],
    "second": ["Error"]
}

## Testing environment protection during test execution

In [56]:
assert "x" not in globals()
mylist = [1, 2, 3]


suite = Suite()


@suite.test
def trial():
    x = 5
    assert x == 5
    mylist.append(4)


assert "x" not in globals()
assert [("trial", [Success])] == [(name, [type(r) for r in rez]) for name, rez in suite.results]
# Accidental globals are obviously not protected.
assert [1, 2, 3, 4] == mylist

In [57]:
for ns in [globals(), locals()]:
    assert "C" not in ns


suite = Suite()


@suite.test
def check_something_about_a_new_class():
    class C:
        def f(self):
            return 5

    assert C().f() == 5


for ns in [globals(), locals()]:
    assert "C" not in ns

## Testing publish/subscribe of results

In [58]:
class TestSubscriber(Subscriber):

    def __init__(self, lr: List[Tuple[str, Result]]):
        self.suite = None
        self._results: List[Tuple[str, Result]] = lr

    def on_add_to_suite(self, suite: Suite) -> None:
        self.suite = suite

    def on_result(self, name_test: str, result: Result) -> None:
        self._results.append((name_test, result))


results: List[Tuple[str, Result]] = []
sub = TestSubscriber(results)
assert sub.suite is None
suite = Suite() | sub
assert sub.suite is suite


@suite.test
def passing():
    pass


assert len(results) == 1
name_last, result_last = results[-1]
assert name_last == "passing"
assert isinstance(result_last, Success)


@suite.test
def error():
    raise RuntimeError("asdf")


assert len(results) == 2
name_last, result_last = results[-1]
assert name_last == "error"
assert isinstance(result_last, Error)
assert str(result_last.value_exc) == "asdf"


@suite.test
def failing():
    fail("There is no why")


assert len(results) == 3
name_last, result_last = results[-1]
assert name_last == "failing"
assert isinstance(result_last, Failure)
assert result_last.reason == "There is no why"

# Generating test run reports

## Report colorizers

In [59]:
class Emphasis(ABC):

    @abstractmethod
    def __call__(self, s: str) -> str:
        raise NotImplementedError()

In [60]:
class Color(Emphasis):

    def __init__(self, fg=None, bg=None, style=None) -> None:
        super().__init__()
        self._fg = fg
        self._bg = bg
        self._style = style

    def __call__(self, s: str) -> str:
        return colors.color(s, fg=self._fg, bg=self._bg, style=self._style)

In [61]:
assert Color(fg="red", bg="blue", style="bold")("asdf") == '\x1b[31;44;1masdf\x1b[0m'

In [62]:
class Plain(Emphasis):

    def __call__(self, s: str) -> str:
        return s

In [63]:
assert Plain()("asdf") == "asdf"

In [64]:
class Colorizer:

    def __init__(self, important: Emphasis, trivial: Emphasis, failure: Emphasis, error: Emphasis) -> None:
        self.important: Emphasis = important
        self.trivial: Emphasis = trivial
        self.failure: Emphasis = failure
        self.error: Emphasis = error

In [65]:
def plain() -> Colorizer:
    return Colorizer(Plain(), Plain(), Plain(), Plain())

In [66]:
c = plain()
assert c.important("asdf") == "asdf"
assert c.trivial("asdf") == "asdf"
assert c.failure("asdf") == "asdf"
assert c.error("asdf") == "asdf"

In [67]:
def color(  # noqa
    important: Optional[Emphasis] = None,
    trivial: Optional[Emphasis] = None,
    failure: Optional[Emphasis] = None,
    error: Optional[Emphasis] = None
) -> Colorizer:
    return Colorizer(
        important or Color(style="bold"),
        trivial or Color(fg="white"),
        failure or Color(fg="yellow"),
        error or Color(fg="red")
    )

In [68]:
c = color(important=Color(fg="white"), trivial=Color(fg="black"), failure=Color(bg="blue"), error=Color(fg="green"))
assert c.important("asdf") == Color(fg="white")("asdf")
assert c.trivial("asdf") == Color(fg="black")("asdf")
assert c.failure("asdf") == Color(bg="blue")("asdf")
assert c.error("asdf") == Color(fg="green")("asdf")

## Raising an exception when reporting failures and errors

This can be useful when running all tests after a code modification, especially if running from within a CI system.

In [69]:
class ProblemsEncountered(Exception):
    """Raised (optionally) when a reporting routine must report failures and errors."""

    def __init__(self, num_failures, num_errors):
        plural_failure = "s" if num_failures > 1 else ""
        plural_errors = "s" if num_errors > 1 else ""
        super().__init__(
            f"Problems encountered during testing: {num_failures} failure{plural_failure}, "
            f"{num_errors} error{plural_errors}"
        )
        self.num_failures = num_failures
        self.num_errors = num_errors

In [70]:
import re

try:
    raise ProblemsEncountered(2, 2)
except ProblemsEncountered as err:
    assert re.search("2 failures, 2 errors", str(err))

try:
    raise ProblemsEncountered(1, 0)
except ProblemsEncountered as err:
    assert re.search("1 failure, 0 error", str(err))

In [71]:
def raise_on_error(suite: Suite) -> None:
    num_failures = 0
    num_errors = 0
    for _, rez in suite.results:
        for r in rez:
            if not r.is_success():
                if r.is_failure():
                    num_failures += 1
                else:
                    num_errors += 1
    if num_failures > 0 or num_errors > 0:
        raise ProblemsEncountered(num_failures, num_errors)

In [72]:
def make_suite_with_two_tests():
    suite = Suite()
    for name in ["one", "two"]:
        @suite.test(name=f"passing-{name}", args=(name,))
        def passing(a):
            pass

    return suite

In [73]:
suite = make_suite_with_two_tests()

try:
    raise_on_error(suite)
except ProblemsEncountered:
    assert False

In [74]:
suite = make_suite_with_two_tests()


@suite.test
def failure():
    fail()


try:
    raise_on_error(suite)
    assert False
except ProblemsEncountered as err:
    assert err.num_failures == 1
    assert err.num_errors == 0

In [75]:
suite = make_suite_with_two_tests()


@suite.test
def error():
    raise RuntimeError()


try:
    raise_on_error(suite)
    assert False
except ProblemsEncountered as err:
    assert err.num_failures == 0
    assert err.num_errors == 1

## Exhaustive report

In [76]:
TestNameFormatter = Callable[[str, int], str]


def name_all(name_test: str, num_result: int) -> str:
    """
    Test name formatter that puts out the name of a test even when it has
    been run multiple times.
    """
    return name_test


def ladder(name_test: str, num_result: int) -> str:
    """
    Test name formatter that puts out the name of a test only once, even
    if it has been run multiple times.
    """
    if num_result == 0:
        return name_test
    return " " * len(name_test)

In [77]:
def quoter(formatter: TestNameFormatter) -> TestNameFormatter:
    """
    Test name formatter that surrounds the name between double quotes. Not
    meant to be used directly by users of this module.
    """
    def quoter_format(name_test: str, num_result: int) -> str:
        return f"\"{formatter(name_test, num_result)}\""

    return quoter_format

In [78]:
class PolicyReportingProblems:
    """What to do when reporting test results that involve problems (failures and errors)."""
    def __init__(self, label: str) -> None:
        self.label = label

    def __str__(self) -> str:
        return self.label

    def __eq__(self, other: Any) -> bool:
        if not isinstance(other, PolicyReportingProblems):
            return False
        return self.label == other.label


IGNORE = PolicyReportingProblems("ignore")
RAISE = PolicyReportingProblems("raise")

In [79]:
def report_results(  # noqa
    suite: Suite,
    file: TextIOBase = sys.stdout,
    colorizer: Colorizer = color(),
    format_name_test: TestNameFormatter = ladder,
    sep_name_result: str = "\t",
    quote_names: bool = False,
    labels_result_custom: Mapping[type, str] = {},
    on_error: PolicyReportingProblems = IGNORE
) -> None:
    """
    Reports the name and result for each attempt at running a test, without details
    as to issues encountered (failures and errors).

    suite
        Suite of test to write report from.
    file
        File-like object to write report to. Default is standard output.
    colorizer
        Policy for emphasizing the written report.
    format_name_test
        Some tests are run more than once (for instance, for iterative problem solving).
        In a report written for human reading, the repeated naming of a test run more
        than once can feel redundant; it is eliminated by setting this to `ladder`.
        All tests will be named if the formatter used instead is `name_all`.
    sep_name_result
        Separating character used between test name and result label. Default is "\t".
    quote_names
        If True, the test names will be surrounded with double quotes in the output.
    labels_result_custom
        Dictionary of labels to use with different result types, when the default
        labels (*ok* for success, *failed* for failure, *ERROR* for error) should be
        changed. The emphasis for each label is derived from the colorizer.
    on_error
        What to do when reporting results that include problems such as failures and
        errors. If set to RAISE, it will raise a ProblemsEncountered exception, which
        is useful when running the notebook as part of a CI/CD pipeline; otherwise,
        or if set to IGNORE, nothing more is done than writing the report.
        Default is IGNORE.
    """
    len_all_names = [len(name) for name, _ in suite.results]
    if len(len_all_names) == 0:
        return
    len_name_largest = max(len_all_names)

    labels_result = {
        type_result: colorize(labels_result_custom.get(type_result, label_default))
        for type_result, colorize, label_default in [
            (Success, Plain(), "ok"),
            (Failure, colorizer.failure, "failed"),
            (Error, colorizer.error, "ERROR")
        ]
    }

    if quote_names:
        format_name_test = quoter(format_name_test)

    for name, rez in suite.results:
        p_name = f"{name:{len_name_largest}s}"
        for num, r in enumerate(rez):
            print(format_name_test(p_name, num), labels_result[type(r)], sep=sep_name_result, file=file)

    if on_error is RAISE:
        raise_on_error(suite)

In [80]:
suite = Suite()


@suite.test
def first():
    raise RuntimeError()


@suite.test(name="first")
def first2():
    pass


@suite.test
def second():
    fail()


@suite.test
def third():
    pass


@suite.test
def fourth():
    assert False


@suite.test(name="fourth")
def fourth2():
    pass

Visual check: tests `first` and `fourth` are run twice; `second` and `third` only once. The name of each test is written only once. The results are either `ok`, `failed` or `ERROR` and are emphasized as normal for success, yellow for failure and red for error. The results are also lined up cleanly into a second column.

In [81]:
report_results(suite)

first 	ERROR
      	ok
second	failed
third 	ok
fourth	failed
      	ok


Same as above, but with a `RAISE` policy.

In [82]:
try:
    report_results(suite, on_error=RAISE)
    assert False
except ProblemsEncountered as err:
    assert err.num_failures == 2
    assert err.num_errors == 1

first 	ERROR
      	ok
second	failed
third 	ok
fourth	failed
      	ok


Visual check: like previous, except that the test names are written on each line a result is reported.

In [83]:
report_results(suite, format_name_test=name_all)

first 	ERROR
first 	ok
second	failed
third 	ok
fourth	failed
fourth	ok


Visual check: like previous, but CSV-like, with test names quoted.

In [84]:
report_results(suite, format_name_test=name_all, quote_names=True, sep_name_result=",", colorizer=plain())

"first ",ERROR
"first ",ok
"second",failed
"third ",ok
"fourth",failed
"fourth",ok


In [85]:
suite_all_passed = Suite()


@suite_all_passed.test
def first():
    pass


@suite_all_passed.test
def second():
    pass

Visual check: two results, both `ok`. No exception raised.

In [86]:
report_results(suite_all_passed, on_error=RAISE)

first 	ok
second	ok


In [87]:
suite_empty = Suite()

Visual check: nothing written.

In [88]:
report_results(suite_empty)

## Test result summary

In [89]:
def summarize_results(  # noqa
    suite: Suite,
    file: Optional[TextIOBase] = sys.stdout,
    colorizer: Colorizer = color(),
    sep: str = ", ",
    on_error: PolicyReportingProblems = IGNORE
) -> Dict[type, int]:
    """
    Writes a very short summary of a test run, counting the number of each result obtained.

    suite
        Suite of test to write report from.
    file
        File-like object to write report to. Default is standard output.
    colorizer
        Policy for emphasizing the written report.
    sep
        Separation string between the labeled numbers of results. Default is ", "
    on_error
        What to do when reporting results that include problems such as failures and
        errors. If set to RAISE, it will raise a ProblemsEncountered exception, which
        is useful when running the notebook as part of a CI/CD pipeline; otherwise,
        or if set to IGNORE, nothing more is done than writing the report.
        Default is IGNORE.
    """
    summary = {t: 0 for t in [Success, Failure, Error]}
    for _, rez in suite.results:
        for r in rez:
            summary[type(r)] += 1

    if file is not None:
        print(
            f"{summary[Success]} passed",
            (colorizer.failure if summary[Failure] > 0 else colorizer.trivial)(f"{summary[Failure]} failed"),
            (colorizer.error if summary[Error] > 0 else colorizer.trivial)(f"{summary[Error]} raised an error"),
            file=file,
            sep=sep
        )

    if on_error == RAISE:
        raise_on_error(suite)
    return summary

Visual check: should show 3 tests passed, 2 failures (in yellow), 1 error (in red).

In [90]:
summarize_results(suite)

3 passed, 2 failed, 1 raised an error


{__main__.Success: 3, __main__.Failure: 2, __main__.Error: 1}

Same, but raising an exception.

In [91]:
try:
    summarize_results(suite, on_error=RAISE)
    assert False
except ProblemsEncountered as err:
    assert err.num_failures == 2
    assert err.num_errors == 1

3 passed, 2 failed, 1 raised an error


Check visually that nothing is written.

In [92]:
assert summarize_results(suite, file=None) == {
    Success: 3,
    Failure: 2,
    Error: 1
}

In [93]:
suite_all_passed = Suite()


@suite_all_passed.test
def first():
    pass


@suite_all_passed.test
def second():
    pass

Check visually that report is fine. Should show 2 tests passed, and failed and errors labeled in a subdued color.

In [94]:
summarize_results(suite_all_passed)

2 passed, 0 failed, 0 raised an error


{__main__.Success: 2, __main__.Failure: 0, __main__.Error: 0}

Check visually that report is fine. Should show 0 for each type of test.

In [95]:
summarize_results(suite_empty)

0 passed, 0 failed, 0 raised an error


{__main__.Success: 0, __main__.Failure: 0, __main__.Error: 0}

## Detailed report of issues encountered

### Printing one frame for a result's associated traceback

In [96]:
def get_id_frame(frame: Frame) -> Sequence[str]:
    return ([f"Function {frame.function}"] if frame.function else []) + [frame.name_file_pretty(True), f"Line {frame.num_line}"]

In [97]:
def print_frame(  # noqa
    frame: Frame,
    file: TextIOBase = sys.stdout,
    colorizer: Colorizer = color(),
    lines_context: int = 3
) -> None:
    """
    Writes up a single stack frame report.

    frame
        Stack frame to report on.
    file
        File-like object to write report to. Default is standard output.
    colorizer
        Policy for emphasizing the written report.
    lines_context
        Number of lines of code to fetch and write up before and after the
        line associated to the stack frame.
    """
    header = (colorizer.trivial if Error.TAG_COMMON in frame.tags else Plain())(" | ".join(get_id_frame(frame)))
    print(header, file=file)
    if Error.TAG_COMMON not in frame.tags:
        context: List[Tuple[int, str]] = frame.context(before=lines_context, after=lines_context)
        if len(context) > 0:
            max_len_num_line = len(str(context[-1][0]))
            for i, line in zip(
                [i for i, _ in context],
                highlight(
                    "\n".join(ln for _, ln in context),
                    lexer=Python3Lexer(),
                    formatter=TerminalFormatter()
                ).split("\n")
            ):
                print(
                    colorizer.trivial(f"{i:{max_len_num_line}d}"),
                    colorizer.trivial("|"),
                    line,
                    sep=" ",
                    file=file
                )

Visual check: show show 3 frame reports. First and third must have appropriate code context (check against the line number), three lines of context around target line max. First frame should have no function name. Second frame report should have subdued color and no code context.

In [98]:
def raiser():
    raise RuntimeError()


def caller():
    raiser()


try:
    caller()
    assert False
except RuntimeError:
    _, _, tb = sys.exc_info()
    for frame, lineno in walk_tb(tb):
        print_frame(
            Frame(
                getframeinfo(frame),
                lineno,
                [Error.TAG_COMMON] if frame.f_code is caller.__code__ else [])
        )
        print()

Code cell 98 | Line 10
 9 | try:
10 |     caller()
11 |     assert False
12 | except RuntimeError:
13 |     _, _, tb = sys.exc_info()

Function caller | Code cell 98 | Line 6

Function raiser | Code cell 98 | Line 2
1 | def raiser():
2 |     raise RuntimeError()
3 | 
4 | 
5 | def caller():



### Detailing one result

In [99]:
def detail_result(  # noqa
    name_test: str,
    result: Error,
    prefix_header: str,
    file: TextIOBase = sys.stdout,
    colorizer: Colorizer = color(),
    lines_context: int = 3,
    print_border: bool = True
) -> None:
    """
    Writes up a report regarding a single test result.

    name_test
        Name of the test the result was gotten for.
    result
        Error-type result to report on.
    prefix_header
        String prepended to the header of the result report.
    file
        File-like object to write report to. Default is standard output.
    colorizer
        Policy for emphasizing the written report.
    lines_context
        Number of lines of code to fetch and write up before and after the
        line associated to the stack frame.
    print_border
        If True (default), prints a border of dashes above the result report.
    """
    header = " ** ".join([
        prefix_header,
        f"Test {colorizer.important(name_test)}",
        {Failure: colorizer.failure, Error: colorizer.error}[type(result)](type(result).__name__)
    ])
    if print_border:
        print("-" * len(colors.strip_color(header)), file=file)
    print(header, file=file)

    if result.is_failure():
        reason = result.reason
        if not reason:
            if result.type_exc is TestFailed:
                reason = "Test marked as failed."
            elif result.type_exc is AssertionError:
                reason = "Assertion failed."
            else:
                reason = "Ill-traced failure meaning. Please take it up to the maintainer."
        print(reason, file=file)

        if result.type_exc is DetailedAssertionError:
            print("\nArguments:\n", file=file)
            args = deepcopy(result.kwops)
            args.update({n: v for n, v in enumerate(result.ops)})
            width_column_name = min(16, max([str(n) for n in args.keys()]))
            for name, value in sorted(list(args.items()), key=lambda p: str(p[0])):
                print(f"    {n:{width_column_name}} = {repr(value)}", file=file)
    else:
        explanation = str(result.value_exc)
        print(
            f"{result.type_exc.__name__}" + (f": {explanation}" if explanation else " (no detail provided)"),
            file=file,
            end="\n\n"
        )
        for frame in result.traceback:  # First frame is always Suite.test, which is irrelevant.
            print_frame(frame, file=file, colorizer=colorizer, lines_context=lines_context)
            print(file=file)

Visual check: header should start with first word **HEY**, label the test as **second**, map it as a *Failure* and provide next the reason of the failure (deliberately marking the test as failed).

In [100]:
name, rez = list(suite.results)[1]
failure = list(rez)[0]
assert isinstance(failure, Failure)
detail_result(name, failure, "HEY")

-----------------------------
HEY ** Test second ** Failure
Test marked as failed.


Visual check:

1. Header should start with first word **HEY**, label the test as **first**, map it as an *Error*.
1. Type of exception raised, with a cue that no further detail had been provided.
1. Backtrace dump.

In [101]:
name, rez = list(suite.results)[0]
error = list(rez)[0]
assert isinstance(error, Error)
detail_result(name, error, "HEY")

--------------------------
HEY ** Test first ** Error
RuntimeError (no detail provided)

Function test | Code cell 49 | Line 73

Function first | Code cell 80 | Line 6
4 | @suite.test
5 | def first():
6 |     raise RuntimeError()
7 | 
8 | 
9 | @suite.test(name="first")



### All results

In [102]:
def detail_issues(  # noqa
    suite: Suite,
    file: TextIOBase = sys.stdout,
    colorizer: Colorizer = color(),
    lines_context: int = 3,
    max_report: int = sys.maxsize,
    on_error: PolicyReportingProblems = IGNORE
) -> None:
    """
    Writes up a report detailing the issues encountered while running the test suite.

    suite
        The test suite.
    file
        The file-like object to write the report to. Default is standard output.
    colorizer
        Color scheme used for emphasizing the various bits of the report.
    lines_context
        Number of lines of context to provide around each line of code involved
        in a reported problem.
    max_report
        Maximum number of problems to report on.
    on_error
        What to do when reporting results that include problems such as failures and
        errors. If set to RAISE, it will raise a ProblemsEncountered exception, which
        is useful when running the notebook as part of a CI/CD pipeline; otherwise,
        or if set to IGNORE, nothing more is done than writing the report.
        Default is IGNORE.
    """
    summary = summarize_results(suite, file=None)
    num_problems = summary[Failure] + summary[Error]
    num_remaining: int = -1
    if num_problems == 0:
        if summary[Success] == 0:
            print("No test run.", file=file)
        else:
            print(f"All {summary[Success]} tests passed. No failure nor error encountered.", file=file)
    else:
        index = 1
        for name, rez in suite.results:
            if num_remaining < 0:
                for r in rez:
                    if not r.is_success():
                        detail_result(name, r, f"# {index}/{num_problems}", lines_context=lines_context, file=file)
                        print(file=file)

                        if index >= max_report:
                            num_remaining = num_problems - index
                            break
                        index += 1

    if num_remaining > 0:
        print(
            colorizer.important(
                f"... plus {num_remaining} other issue{'s' if num_remaining > 1 else ''}."
            ),
            file=file
        )
    if on_error == RAISE:
        raise_on_error(suite)

Visual check: 3 problem reports expected. First one is for an undetailed error (`RuntimeError`), with a following backtrace dump. The first frame report of this dump (`Function test | `...) is in a subdued color. Second and third are for failures. 

In [103]:
detail_issues(suite)

----------------------------
# 1/3 ** Test first ** Error
RuntimeError (no detail provided)

Function test | Code cell 49 | Line 73

Function first | Code cell 80 | Line 6
4 | @suite.test
5 | def first():
6 |     raise RuntimeError()
7 | 
8 | 
9 | @suite.test(name="first")


-------------------------------
# 2/3 ** Test second ** Failure
Test marked as failed.

-------------------------------
# 3/3 ** Test fourth ** Failure
Assertion failed.



Visual check: show only the report for the error, and mention that two more problems have yet to be reported. Also, code context is only one line around the target line, making for a 3-line code blurb.

In [104]:
detail_issues(suite, max_report=1, lines_context=1)

----------------------------
# 1/3 ** Test first ** Error
RuntimeError (no detail provided)

Function test | Code cell 49 | Line 73

Function first | Code cell 80 | Line 6
5 | def first():
6 |     raise RuntimeError()


... plus 2 other issues.


Same, but with exception raising.

In [105]:
try:
    detail_issues(suite, max_report=1, lines_context=1, on_error=RAISE)
    assert False
except ProblemsEncountered as err:
    assert err.num_failures == 2
    assert err.num_errors == 1

----------------------------
# 1/3 ** Test first ** Error
RuntimeError (no detail provided)

Function test | Code cell 49 | Line 73

Function first | Code cell 80 | Line 6
5 | def first():
6 |     raise RuntimeError()


... plus 2 other issues.


Visual check: now show the error and the first failure, with 0 code context (so only the target line is shown). The final message about remaining issues is singular.

In [106]:
detail_issues(suite, max_report=2, lines_context=0)

----------------------------
# 1/3 ** Test first ** Error
RuntimeError (no detail provided)

Function test | Code cell 49 | Line 73

Function first | Code cell 80 | Line 6
6 |     raise RuntimeError()


-------------------------------
# 2/3 ** Test second ** Failure
Test marked as failed.

... plus 1 other issue.


Visual check: two tests passed, no failure nor error.

In [107]:
detail_issues(suite_all_passed)

All 2 tests passed. No failure nor error encountered.


Visual check: no test has been run.

In [108]:
detail_issues(suite_empty)

No test run.


# On-the-fly result reporting

In [109]:
class Report(Subscriber):
    """
    Test suite subscriber that reports on test results on-the-fly. When a test
    does not succeed, details on the failure can optionally be provided. See
    documentation on class `Suite` to get an example on the usage of this
    plug-in.

    file
        File-like object where the test results are reported. Default is
        standard output.
    verbose
        If True, the feedback on test results contains traceback information
        when problems are encountered.
    file
        File-like object where the feedback is put out.
    colorizer
        Policy on how to emphasize the feedback output.
    lines_context
        Number of lines of code to provide as context in traceback frames
        around the line of code at the nexus of an issue.
    """
    def __init__(
        self,
        verbose: bool = True,
        file: TextIOBase = sys.stdout,
        colorizer: Colorizer = color(),
        lines_context: int = 3
    ) -> None:
        super().__init__()
        self._file = file
        self._verbose = verbose
        self._colorizer = colorizer
        self._lines_context = lines_context

    def on_result(self, name_test: str, result: Result) -> None:
        name_test = self._colorizer.important(name_test)
        if result.is_success():
            print(f"Test {name_test} passed.", file=self._file)
        else:
            if self._verbose:
                detail_result(
                    name_test,
                    result,
                    "Issue",
                    self._file,
                    self._colorizer,
                    self._lines_context,
                    print_border=False
                )
            else:
                frame_relevant = result.traceback[-1]
                details_frame = get_id_frame(frame_relevant)
                if result.is_failure():
                    label = self._colorizer.failure(f"Failed test {name_test}")
                    print(f"{label}: {result.reason} | {' | '.join(details_frame)}", file=self._file)
                else:
                    label = self._colorizer.error(f"Error occured during test {name_test}")
                    value_exc = ""
                    if str(result.value_exc):
                        value_exc = f" -- {str(result.value_exc)}"
                    print(f"{label}: {result.type_exc.__name__}{value_exc}", file=self._file)
                    print_frame(
                        frame_relevant,
                        file=self._file,
                        colorizer=self._colorizer,
                        lines_context=self._lines_context
                    )

In [110]:
def exercise_suite(reporter):
    suite = Suite() | reporter

    @suite.test
    def passing():
        pass

    @suite.test
    def failing():
        assert 1 == 0, "one vs. zero"

    @suite.test
    def failing_no_reason():
        assert 1 == 0

    @suite.test
    def error_raising():
        raise RuntimeError("Some error")

    @suite.test
    def error_raising_no_str():
        raise RuntimeError()

Visual check: first and second tests pass, third test fails, fourth test raises an error, last two tests pass. Failure and error are tersely described, only the most relevant stack frame is provided.

In [111]:
exercise_suite(Report(verbose=False))

Test passing passed.
Failed test failing: one vs. zero | Function failing | Code cell 110 | Line 10
Failed test failing_no_reason:  | Function failing_no_reason | Code cell 110 | Line 14
Error occured during test error_raising: RuntimeError -- Some error
Function error_raising | Code cell 110 | Line 18
16 |     @suite.test
17 |     def error_raising():
18 |         raise RuntimeError("Some error")
19 | 
20 |     @suite.test
21 |     def error_raising_no_str():
Error occured during test error_raising_no_str: RuntimeError
Function error_raising_no_str | Code cell 110 | Line 22
20 |     @suite.test
21 |     def error_raising_no_str():
22 |         raise RuntimeError()


Visual check: like previous, but with full detail of each non-success.

In [112]:
exercise_suite(Report())

Test passing passed.
Issue ** Test failing ** Failure
one vs. zero
Issue ** Test failing_no_reason ** Failure
Assertion failed.
Issue ** Test error_raising ** Error
RuntimeError: Some error

Function test | Code cell 49 | Line 73

Function error_raising | Code cell 110 | Line 18
16 |     @suite.test
17 |     def error_raising():
18 |         raise RuntimeError("Some error")
19 | 
20 |     @suite.test
21 |     def error_raising_no_str():

Issue ** Test error_raising_no_str ** Error
RuntimeError (no detail provided)

Function test | Code cell 49 | Line 73

Function error_raising_no_str | Code cell 110 | Line 22
20 |     @suite.test
21 |     def error_raising_no_str():
22 |         raise RuntimeError()



# Using a cell magic to write tests

The current decorator approach involves little boilerplate, but it can be reduced further nonetheless. Let's use a cell magic, in which all code written is wrapped into an ad hoc test function. Let's use a subscriber to register this cell magic.

In [113]:
class Magic(Subscriber):

    def __init__(self, name_magic: str = "test") -> None:
        self._name_magic = name_magic

    def on_add_to_suite(self, suite):
        suite._fns_common.append(run_test_from_cell)
        ipython = get_ipython()
        if ipython and self._name_magic:
            register_cell_magic(self._name_magic)(lambda line, cell: test_cell(suite, line, cell))


def test_cell(suite: Suite, line: str, cell: Optional[str]) -> None:
    """
    Runs a test written using a cell magic.
    """
    line = line.strip()
    if not line:
        raise ValueError("Please provide a title for the test (right after the cell magic invocation).")
    cell = (cell or "").strip()
    if not cell:
        raise ValueError("There is no test to execute! Please write some code in there.")

    suite.test(run_test_from_cell, name=line, args=(cell,))


def run_test_from_cell(cell: str) -> None:
    """
    Executes the body of a cell, in context of the execution of a test.
    """
    code_source = "\n" + cell
    ipython = get_ipython()
    name_cell = ipython.compile.cache(code_source)
    code = compile(code_source, name_cell, "exec")
    exec(code, ipython.user_global_ns, locals())

## Tests

### Simple usage of the magic

In [114]:
suite = Suite() | Magic()

In [115]:
%%test First test, passing
assert True

In [116]:
%%test Second test, failing
fail()

In [117]:
%%test Third test, raising an error
raise RuntimeError()

In [118]:
def my_function(x, y):
    return x * y + x + y

In [119]:
%%test function in my notebook
assert my_function(4, 5) == 29

In [120]:
assert {name: [r["type"] for r in rez] for name, rez in suite.as_dict().items()} == {
    "First test, passing": ["Success"],
    "Second test, failing": ["Failure"],
    "Third test, raising an error": ["Error"],
    "function in my notebook": ["Success"]
}

Ensure even with the cell magic environment is not polluted.

In [121]:
for name in ["x", "f", "C"]:
    for ns in [globals(), locals(), get_ipython().user_ns]:
        assert name not in ns

In [122]:
%%test Add x, f and C
x = 5

def f():
    return 5


class C():
    
    def f(self):
        return 5
    
    
assert f() == x
assert C().f() == f()

In [123]:
for name in ["x", "f", "C"]:
    for ns in [globals(), locals(), get_ipython().user_ns]:
        try:
            assert name not in ns
        except AssertionError:
            print(f"Name {name} still defined.")
            raise

### Detailing one result, obtained by testing with cell magic

In [124]:
suite_magic = Suite() | Magic("test_magic")

In [125]:
%%test_magic the label
x = 5
assert x == 5
fail()

Visual check: header should start with first word **Head**, label the test as **the label** and map it as a *Failure*. Its frame report shows three subdued frames without code context, sandwiching a frame showing the call to the `fail()` function (line 16) that tripped the failure.

In [126]:
name, rez = list(suite_magic.results)[0]
failure = list(rez)[0]
assert isinstance(failure, Failure)
detail_result(name, failure, "Head")

---------------------------------
Head ** Test the label ** Failure
Test marked as failed.


# Documentation index

Let's compose a docstring for the `jupytest` module (which we will export out of bits of this notebook). This docstring will act as a reference documentation index.

# Exportation of testing tools to `jupytest.py` module

In [127]:
import os
import os.path
import shutil

if os.path.isdir("jupytest"):
    print("Clean up previous")
    shutil.rmtree("jupytest")
os.makedirs("jupytest")

Clean up previous


In [128]:
import nbformat
with open("jupytest.ipynb", "r", encoding="utf-8") as file_notebook:
    nb = nbformat.read(file_notebook, nbformat.NO_CONVERT)

In [129]:
with open("jupytest/__init__.py", "w", encoding="utf-8") as file_module:
    print(
        "\n\n".join(
            [
                "\n\n".join(
                    f'"""\n{cell.source}\n"""'
                    for cell in nb.cells
                    if cell.cell_type == 'raw' and "header" in cell.metadata.get("tags", [])
                )
            ] +
            [
                cell.source
                for cell in nb.cells
                if cell.cell_type == 'code' and "module" in cell.metadata.get("tags", [])
            ]
        ),
        file=file_module,
        end=""
    )

In [130]:
import importlib

# Force the reload of the new jupytest module. If running from a fresh
# kernel, the reload is spurious but innocuous.
import jupytest
importlib.reload(jupytest)
None

Check that the exported module has all the tools we defined.

In [131]:
for construct in [
    "Result",
    "Success",
    "Frame",
    "Error",
    "Failure",
    "TestFailed",
    "fail",
    "Subscriber",
    "Suite",
    "Emphasis",
    "Color",
    "Plain",
    "Colorizer",
    "color",
    "plain",
    "TestNameFormatter",
    "ladder",
    "name_all",
    "quoter",
    "ProblemsEncountered",
    "raise_on_error",
    "PolicyReportingProblems",
    "IGNORE",
    "RAISE",
    "report_results",
    "summarize_results",
    "print_frame",
    "detail_result",
    "detail_issues",
    "Report",
    "join_args",
    "Explanation",
    "DetailedAssertionError",
    "assert_",
    "_assess_sequence_2more",
    "_assessor_sequence",
    "same",
    "eq",
    "diff",
    "between",
    "approx",
    "belong_to",
    "is_any_of"
]:
    assert hasattr(jupytest, construct), f"Have not got construct {construct}"

Visual check: ensure the documentation for the `jupytest` module itself corresponds to the `header` cell.

In [132]:
print(help(jupytest))

Help on package jupytest:

NAME
    jupytest - Unit and integration testing in a notebook

DESCRIPTION
    *** Building and running suites of tests ***
    
    class Suite
        Method test (context manager)
    Function fail
    
    *** Reporting test results ***
    
    Function report_results
    Function summarize_results
    Function detail_issues
    Class Report (used as a subscriber plug-in to class Suite)
    
    *** Delving deeper into test results (going beyond the tools described above) ***
    
    Class Suite
        Property results
    Class Result
        Sub-class Success
        Sub-class Error
            Sub-class Failure
        Method is_success
        Method is_failure
    Class Frame
    
    *** Customizing result reporting ***
    
    Class Colorizer
        Function plain
        Function color
    Class Emphasis
        Sub-class Plain
        Sub-class Color
    Type TestNameFormatter
        Function ladder
        Function name_all
        Functi